--- BOLIN PUT YOUR CODE HERE ----
<br>
--- Data preprocessing and RQ1 ----
<br>
The following script demonstrates how to automate the download of data using URI from internet to HDFS.(Here the green taxi data of 2019 is used as an example.)

In [ ]:
'''
I wrote this script to help automate the process of downloading data from the Internet into HDFS
and to avoid storing too much data in NFS simutaneously.
Execute this program in spark environment, or run it in our own IDE but you should have internet access to the UT spark framework and the path should also be changed accordingly.
'''

import os
import wget
import subprocess

#Put the urls from which you pull data into NFS system in this list
urls = ["https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-01.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-02.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-03.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-04.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-05.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-06.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-07.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-08.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-09.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-10.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-11.parquet",
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-12.parquet"
        ]

my_nfs_path = "/home/s3022455"
my_hdfs_path = "/user/s3022455/taxiData/Green/2019"

for url in urls:
    #Use wget to download data into my NFS system. Here I use my own path. If you wanna customizs it you should set it above as your own path.
    filename = wget.download(url, out=my_nfs_path)

    #Transfer the data from my NFS to my HDFS
    subprocess.check_call(['hdfs', 'dfs', '-put', filename, my_hdfs_path])

    #Delete the data in NFS
    os.remove(filename)

The following script returns the first 20 most frequent drop-off locations of green taxi in 2019. For specifically extracting weekday or weekday data, just add a filter method like what the comment shows.

In [ ]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import split, col, trim, explode, desc, count
from pyspark.sql.types import DoubleType

sc = SparkContext(appName = "Testing")
spark = SparkSession(sc)
sc.setLogLevel("ERROR")

#df1 = spark.read.json(sc.parallelize([parse_green]))
file_path_list = ["/path/of/my/file1", "/path/of/my/file2", ..."/path/of/my/fileN"]

#Line 19-28 is to unify "ehail_fee" in all 2019 green taxi dataset in DoubleType, or the merge of parquet files will fail.
#If all the files have the same data type on each column, you can directly merge it from the code of line 31.
dfs = []

for file_path in file_path_list:
    df = spark.read.parquet(file_path)
    df = df.withColumn("ehail_fee", col("ehail_fee").cast(DoubleType()))
    dfs.append(df)

df1 = dfs[0]
for df in dfs[1:]:
    df1 = df1.union(df)

df1 = spark.read.parquet(*file_path_list, mergeSchema=True)

#To tag whether this trip is on weekday or on weekend, very useful I have to say.
df2 = df1.withColumn("weekend", ((dayofweek("lpep_pickup_datetime")==1)|(dayofweek("lpep_pickup_datetime")==7)).cast("int"))

df3 = df1.drop("mta_tax", "improvement_surcharge", "VendorID")

#Turn the datatype of values in "dolocationid" into array so that explode method can be applied
df4 = df3.withColumn("DOLocationID", trim(col("DOLocationID"))).withColumn("DOLocationID", split(col("DOLocationID"), ","))

#df5 = df4.filter(df3.weekend == 1 or 0)

df5 = df4.select(explode("DOLocationID").alias("DOLocationID"))

df6 = df5.groupBy("DOLocationID").agg(count(col("*")).alias("count"))

df7 = df6.orderBy(desc("count")).show(20)

df8 = df7.limit(20)

df8.write.csv("/user/s3022455/Green")

"""
Green taxi 2019 example
+------------+------+
|DOLocationID| count|
+------------+------+
|          74|230014|
|          42|214976|
|          41|190330|
|          75|166268|
|         129|157254|
|           7|155140|
|         166|128331|
|         181|106210|
|          82|104244|
|         223| 96710|
|          95| 96558|
|         236| 96190|
|         244| 91202|
|         238| 90936|
|          61| 90322|
|         116| 85612|
|          97| 84012|
|          49| 72124|
|         138| 71428|
|         226| 70451|
+------------+------+
"""


--- HAKAN PUT YOUR CODE HERE ----
<br>
--- RQ2 ----

In [ ]:
## Hakan's code



from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, concat, lit, expr,format_string, hour, asc,input_file_name

sc = SparkContext(appName = 'rqProject')
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

#for Green Data: columnsToKeep = ['VendorID','lpep_pickup_datetime','lpep_dropoff_datetime']
columnsToKeep = ['hvfhs_license_num','dispatching_base_num', 'request_datetime','pickup_datetime']
hdfs_directory_path = '/user/s3022455/'

file_paths = [
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-01.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-02.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-03.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-04.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-05.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-06.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-07.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-08.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-09.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2023/fhvhv_tripdata_2023-10.parquet'
]


df = spark.read.parquet(*file_paths)
df2 = df.select(columnsToKeep)
dfWithFileName = df2.withColumn('hvfhv-2019', input_file_name())
#for Green Data: df3 = df2.withColumn('hour_range', expr("concat(date_format(lpep_pickup_datetime, 'HH:00:00'), ' - ', date_format(lpep_pickup_datetime, 'HH:59:59'))"))
df3 = df2.withColumn('timestamp', expr("concat(date_format(request_datetime, 'HH:00:00'), ' - ', date_format(request_datetime, 'HH:59:59'))"))
df4 = df3.groupBy('timestamp').count().sort(col('timestamp'))
df4.write.csv('/user/s3105555/project', header=True, mode='overwrite')

'''
KEEP LINES BELOW, It works for a single file
df = spark.read.parquet(/user/s3022455/taxiData/Green/2019/green_tripdata_2019-01.parquet)
df2 = df.select(columnsToKeep)
df3 = df2.withColumn('hour_range', expr("concat(date_format(lpep_pickup_datetime, 'HH:00:00'), ' - ', date_format(lpep_pickup_datetime, 'HH:59:59'))"))
df4 = df3.groupBy('hour_range').count().sort(col('hour_range'))
df4.write.csv('/user/s3105555/project', mode = 'overwrite')
'''

'''
2019
file_paths = [
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-01.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-02.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-03.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-04.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-05.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-06.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-07.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-08.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-09.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-10.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-11.parquet',
    hdfs_directory_path + 'taxiData/Green/2019/green_tripdata_2019-12.parquet'
]

2020
file_paths = [
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-01.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-02.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-03.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-04.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-05.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-06.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-07.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-08.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-09.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-10.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-11.parquet',
    hdfs_directory_path + 'taxiData/Green/2020/green_tripdata_2020-12.parquet'
]

2021
file_paths = [
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-01.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-02.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-03.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-04.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-05.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-06.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-07.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-08.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-09.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-10.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-11.parquet',
    hdfs_directory_path + 'taxiData/Green/2021/green_tripdata_2021-12.parquet'
]

2022
file_paths = [
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-01.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-02.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-03.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-04.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-05.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-06.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-07.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-08.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-09.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-10.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-11.parquet',
    hdfs_directory_path + 'taxiData/Green/2022/green_tripdata_2022-12.parquet'
]

2023
file_paths = [
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-01.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-02.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-03.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-04.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-05.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-06.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-07.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-08.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-09.parquet',
    hdfs_directory_path + 'taxiData/Green/2023/green_tripdata_2023-10.parquet'
]


'''

'''
hvfhv
2019
file_paths = [
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-02.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-03.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-04.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-05.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-06.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-07.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-08.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-09.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-10.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-11.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2019/fhvhv_tripdata_2019-12.parquet'
]

2020

file_paths = [
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-01.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-02.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-03.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-04.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-05.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-06.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-07.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-08.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-09.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-10.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-11.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2020/fhvhv_tripdata_2020-12.parquet'
]

2021
file_paths = [
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-01.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-02.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-03.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-04.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-05.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-06.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-07.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-08.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-09.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-10.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-11.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2021/fhvhv_tripdata_2021-12.parquet'
]

2022
file_paths = [
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-01.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-02.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-03.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-04.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-05.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-06.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-07.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-08.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-09.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-10.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-11.parquet',
    hdfs_directory_path + 'taxiData/HVFHV/2022/fhvhv_tripdata_2022-12.parquet'
]
'''

--- YAWEN AND WAN PUT YOUR CODE HERE ----
<br>
--- RQ3 ----
<br>
Research Question 3, this code was originally written and run on the cluster using Nano code editor as well as all the libraries mentioned below

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, mean, count, min, max
import matplotlib.pyplot as plt

# Initialize a Spark session
spark = SparkSession.builder.appName("GreenTaxiHVFHVAnalysis").getOrCreate()

# Set the log level to ERROR
spark.sparkContext.setLogLevel("ERROR")

# Define the base paths for the green taxi and HVFHV data
base_path_green = "/user/s3022455/taxiData/Green/"
base_path_hvfhv = "/user/s3022455/taxiData/HVFHV/"

# Initialize empty DataFrames for the green taxi and HVFHV data by reading the first available dataset
green_taxi_df= spark.read.parquet(f"{base_path_green}2019/*.parquet")
hvfhv_df = spark.read.parquet(f"{base_path_hvfhv}2019/*.parquet")

# Load data from each subsequent year for both datasets and union with the initial Dataframe
for year in range(2019, 2023):  # Up to December 2022
    # Read all Parquet files for the current year for green taxi
    yearly_data_green = spark.read.parquet(f"{base_path_green}2019/*.parquet")
    # Read all Parquet files for the current year for HVFHV
    yearly_data_hvfhv = spark.read.parquet(f"{base_path_hvfhv}2019/*.parquet")

    # Union the yearly data with the main DataFrames
    green_taxi_df = green_taxi_df.unionByName(yearly_data_green, allowMissingColumns=True)
    hvfhv_df = hvfhv_df.unionByName(yearly_data_hvfhv, allowMissingColumns=True)

# Show the first few rows of the green_taxi_df DataFrame
print("Green Taxi DataFrame:")
green_taxi_df.show()

# Show the first few rows of the hvfhv_df DataFrame
print("HVFHV DataFrame:")
hvfhv_df.show()

# Select only the 'trip_distance' and 'trip_miles'  column and convert miles to kilometers if necessary
green_taxi_df = green_taxi_df.select(
    (col("trip_distance") * 1.60934).alias("trip_distance_km")
)

hvfhv_df = hvfhv_df.select(
    (col("trip_miles") * 1.60934).alias("trip_distance_km")
)

# REMOVING OUTLIERS

# Define reasonable limits for trip distances
MIN_DISTANCE = 0  # minimum distance in km
MAX_DISTANCE = 500  # maximum distance in km

# Clean and filter Green Taxi DataFrame
green_taxi_df_clean = green_taxi_df.filter(
    (col("trip_distance_km") > MIN_DISTANCE) &
    (col("trip_distance_km") <= MAX_DISTANCE)
)

# Clean and filter HVFHV DataFrame
hvfhv_df_clean = hvfhv_df.filter(
    (col("trip_miles") > MIN_DISTANCE) &
    (col("trip_miles") <= MAX_DISTANCE)
)

# Define the distance thresholds for categorizing trips
short_distance_threshold = 5  # km
long_distance_threshold = 15  # km

# Create a new categorical column 'trip_type_category' in both DataFrames
green_taxi_df = green_taxi_df_clean.withColumn(
    'trip_type_category',
    when(col('trip_distance_km') < short_distance_threshold, 'short')
    .when(col('trip_distance_km') <= long_distance_threshold, 'medium')
    .otherwise('long')
)

hvfhv_df = hvfhv_df_clean.withColumn(
    'trip_type_category',
    when(col('trip_distance_km') < short_distance_threshold, 'short')
    .when(col('trip_distance_km') <= long_distance_threshold, 'medium')
    .otherwise('long')
)

# Check for null values in trip_distance for green taxis
green_taxi_nulls = green_taxi_df.filter(col("trip_distance").isNull()).count()
print(f"Number of null values in trip_distance for green taxis: {green_taxi_nulls}")

# Remove rows with null values in trip_distance for green taxis
green_taxi_df = green_taxi_df.filter(col("trip_distance").isNotNull())

# Check for null values in trip_distance for HVFHV
hvfhv_nulls = hvfhv_df.filter(col("trip_miles").isNull()).count()
print(f"Number of null values in trip_distance for HVFHV: {hvfhv_nulls}")

# Remove rows with null values in trip_distance for HVFHV
hvfhv_df = hvfhv_df.filter(col("trip_miles").isNotNull())

# DATA SEGMENTATION

# Segment the data based on the trip type categories
green_taxi_short_trips = green_taxi_df.filter(col('trip_type_category') == 'short')
green_taxi_medium_trips = green_taxi_df.filter(col('trip_type_category') == 'medium')
green_taxi_long_trips = green_taxi_df.filter(col('trip_type_category') == 'long')

hvfhv_short_trips = hvfhv_df.filter(col('trip_type_category') == 'short')
hvfhv_medium_trips = hvfhv_df.filter(col('trip_type_category') == 'medium')
hvfhv_long_trips = hvfhv_df.filter(col('trip_type_category') == 'long')

# Show the count for each segment
print("Green Taxi Short Trips Count:", green_taxi_short_trips.count())
print("Green Taxi Medium Trips Count:", green_taxi_medium_trips.count())
print("Green Taxi Long Trips Count:", green_taxi_long_trips.count())
print("HVFHV Short Trips Count:", hvfhv_short_trips.count())
print("HVFHV Medium Trips Count:", hvfhv_medium_trips.count())
print("HVFHV Long Trips Count:", hvfhv_long_trips.count())

# DESCRIPTIVE ANALYSIS

green_taxi_df.show()

# Calculate overall mean and median trip distance for Green Taxis
green_taxi_overall_stats = green_taxi_df.agg(
    F.count("trip_distance_km").alias("total_num_trips"),
    F.mean("trip_distance_km").alias("overall_mean_distance"),
    F.percentile_approx("trip_distance_km", 0.5).alias("overall_median_distance")
)

# Calculate overall mean and median trip distance for HVFHV
hvfhv_overall_stats = hvfhv_df.agg(
    F.count("trip_distance_km").alias("total_num_trips"),
    F.mean("trip_distance_km").alias("overall_mean_distance"),
    F.percentile_approx("trip_distance_km", 0.5).alias("overall_median_distance")
)

# Find the min and max values for the Green Taxi DataFrame
green_min_max = green_taxi_df.agg(
    min("trip_distance_km").alias("min_distance"),
    max("trip_distance_km").alias("max_distance")
).collect()

green_min_distance = green_min_max[0]['min_distance']
green_max_distance = green_min_max[0]['max_distance']

print(f"Green Taxi - Min Distance: {green_min_distance}, Max Distance: {green_max_distance}")

# Find the min and max values for the HVFHV DataFrame
hvfhv_min_max = hvfhv_df.agg(
    min("trip_distance_km").alias("min_distance"),
    max("trip_distance_km").alias("max_distance")
).collect()

hvfhv_min_distance = hvfhv_min_max[0]['min_distance']
hvfhv_max_distance = hvfhv_min_max[0]['max_distance']

print(f"HVFHV - Min distance: {hvfhv_min_distance}, Max distance: {hvfhv_max_distance}")

# Show the overall stats for Green Taxis
print("Overall Green Taxi Stats:")
green_taxi_overall_stats.show()

# Show the overall stats for HVFHV
print("Overall HVFHV Stats:")
hvfhv_overall_stats.show()

# PLOTTING

# get the counts for each category
green_short_count = green_taxi_short_trips.count()
green_medium_count = green_taxi_medium_trips.count()
green_long_count = green_taxi_long_trips.count()

hvfhv_short_count = hvfhv_short_trips.count()
hvfhv_medium_count = hvfhv_medium_trips.count()
hvfhv_long_count = hvfhv_long_trips.count()

# create lists of counts and labels for the pie charts
green_counts = [green_short_count, green_medium_count, green_long_count]
hvfhv_counts = [hvfhv_short_count, hvfhv_medium_count, hvfhv_long_count]
labels = ['Short', 'Medium', 'Long']
colors = ['#ff9999', '#66b3ff', '#99ff99']

# Green Taxi Pie Chart
fig1, ax1 = plt.subplots()
ax1.pie(green_counts, colors=colors, labels=labels, autopct='%.2f%%', startangle=90)
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
ax1.axis('equal')
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.title('Green Taxi - Segmentation of Trips', pad=20)
plt.savefig('/home/s2819201/green_taxi_trip_segmentation.png', bbox_inches='tight', dpi=300)
plt.close(fig)  # Close the figure to free up memory

# HVFHV Pie Chart
fig1, ax1 = plt.subplots()
ax1.pie(hvfhv_counts, colors=colors, labels=labels, autopct='%.2f%%', startangle=90)
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
ax1.axis('equal')
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.title('HVFHV - Segmentation of Trips', pad=20)
plt.savefig('/home/s2819201/hvfhv_trip_segmentation.png', bbox_inches='tight', dpi=300)
plt.close(fig)  # Close the figure to free up memory

# Define a range for sampling
green_min_sample = 0
green_max_sample = 50
HVFHV_min_sample = 0
HVFHV_max_sample = 50

# Sampling 8% of the data for Green Taxi
green_sample = green_taxi_df.filter(
    (col("trip_distance_km") >= green_min_sample) & (col("trip_distance_km") <= green_max_sample)
).sample(False, 0.08).select('trip_distance_km').rdd.flatMap(lambda x: x).collect()

# Plot the histogram for Green Taxi
plt.figure(figsize=(10, 5))
plt.hist(green_sample, bins=50, color='green', alpha=0.7)

# Lines for threshold, mean, and median
threshold_short = 5  # km for short trips
threshold_long = 15  # km for long trips
green_mean_distance = green_taxi_df.select(mean('trip_distance_km')).collect()[0][0]
green_median_distance = green_taxi_df.approxQuantile('trip_distance_km', [0.5], 0.05)[0]

plt.axvline(threshold_short, color='yellow', linestyle='dashed', linewidth=1, label=f'Short Threshold: {threshold_short} km')
plt.axvline(threshold_long, color='red', linestyle='dashed', linewidth=1, label=f'Long Threshold: {threshold_long} km')
plt.axvline(green_mean_distance, color='blue', linestyle='dashed', linewidth=1, label=f'Mean: {green_mean_distance:.2f} km')
plt.axvline(green_median_distance, color='orange', linestyle='dashed', linewidth=1, label=f'Median: {green_median_distance:.2f} >
plt.title('Green Taxi Trip Distance Distribution')
plt.xlabel('Distance (km)')
plt.ylabel('Frequency')
plt.legend()

#Save the figure for Green Taxi
plt.savefig('/home/s2819201/green_taxi_trip_distance_distribution.png')
plt.close()

# Sampling 8% of the data for HVFHV
hvfhv_sample = hvfhv_df.filter(
    (col("trip_distance_km") >= HVFHV_min_sample) & (col("trip_distance_km") <= HVFHV_max_sample)
).sample(False, 0.08).select('trip_distance_km').rdd.flatMap(lambda x: x).collect()

# Plot the histogram for HVFHV
plt.figure(figsize=(10, 5))
plt.hist(hvfhv_sample, bins=50, color='blue', alpha=0.7)

# Lines for threshold, mean, and median
hvfhv_mean_distance = hvfhv_df.select(mean('trip_distance_km')).collect()[0][0]
hvfhv_median_distance = hvfhv_df.approxQuantile('trip_distance_km', [0.5], 0.05)[0]

plt.axvline(threshold_short, color='yellow', linestyle='dashed', linewidth=1, label=f'Short Threshold: {threshold_short} km')
plt.axvline(threshold_long, color='red', linestyle='dashed', linewidth=1, label=f'Long Threshold: {threshold_long} km')
plt.axvline(hvfhv_mean_distance, color='green', linestyle='dashed', linewidth=1, label=f'Mean: {hvfhv_mean_distance:.2f} km')
plt.axvline(hvfhv_median_distance, color='orange', linestyle='dashed', linewidth=1, label=f'Median: {hvfhv_median_distance:.2f} >

# Add title and labels
plt.title('HVFHV Trip Distance Distribution')
plt.xlabel('Distance (km)')
plt.ylabel('Frequency')
plt.legend()

# Save the figure for HVFHV
plt.savefig('/home/s2819201/hvfhv_trip_distance_distribution.png')
plt.close()

# Stop the Spark Session
spark.stop()

"""
Number of null values in trip_distance for green taxis: 0
Number of null values in trip_distance for HVFHV: 0
Green Taxi Short Trips Count: 19696055
Green Taxi Medium Trips Count: 8376030
Green Taxi Long Trips Count: 2646040
HVFHV Short Trips Count: 619494890
HVFHV Medium Trips Count: 402896060
HVFHV Long Trips Count: 148189580
+------------------+------------------+
|  trip_distance_km|trip_type_category|
+------------------+------------------+
|         1.3840324|             short|
|1.0621644000000001|             short|
|         4.3130312|             short|
|         7.2903102|            medium|
|          1.689807|             short|
|         6.0672118|            medium|
| 6.598293999999999|            medium|
|         12.472385|            medium|
| 5.922371200000001|            medium|
|        11.0078856|            medium|
|1.8507409999999997|             short|
|         0.7885766|             short|
| 5.809717399999999|            medium|
|1.9312079999999998|             short|
|           0.80467|             short|
|           8.85137|            medium|
|         8.0627934|            medium|
|         0.6920162|             short|
|         4.3774048|             short|
|3.4600809999999997|             short|
+------------------+------------------+
only showing top 20 rows

Green Taxi - Min Distance: 0.0160934, Max Distance: 325.247614
HVFHV - Min distance: 0.0016093400000000001, Max distance: 794.0322626
Overall Green Taxi Stats:
+---------------+---------------------+-----------------------+
|total_num_trips|overall_mean_distance|overall_median_distance|
+---------------+---------------------+-----------------------+
|       30718125|    5.858048259059208|              3.4118008|
+---------------+---------------------+-----------------------+

Overall HVFHV Stats:
+---------------+---------------------+-----------------------+
|total_num_trips|overall_mean_distance|overall_median_distance|
+---------------+---------------------+-----------------------+
|     1170580530|    7.633885356374852|              4.6509926|
+---------------+---------------------+-----------------------+

generated plotting files:
green_taxi_trip_distance_distribution.png
green_taxi_trip_segmentation.png
hvfhv_trip_distance_distribution.png
hvfhv_trip_segmentation.png
"""